<center>
    <img src="https://upload.wikimedia.org/wikipedia/commons/a/a8/%D0%9B%D0%9E%D0%93%D0%9E_%D0%A8%D0%90%D0%94.png" width=500px/>
    <font>Python 2023</font><br/>
    <br/>
    <br/>
    <b style="font-size: 2em">Исключения. Менеджеры контекста</b><br/>
    <br/>
    <font>Сапожников Денис</font><br/>
</center>

# Исключения

Часто в программах что-то идёт не так. Если ничего не предпринимать, они ломаются.

In [1]:
def parse_tskv(tskv: str) -> dict[str, int]:
    """Parse tskv string"""
    kvpairs = (keyvalue.split('=') for keyvalue in tskv.strip().split('\t'))
    return {k: int(v) for k, v in kvpairs}

log = [
    'banner_id=1\tshows=10\tclicks=1',
    'banner_id=2\tshows=15\tclicks=2',
    'banner_id=3\tshows=\tclicks=1',  # empty shows
]

for row in log:
    print(parse_tskv(row))

{'banner_id': 1, 'shows': 10, 'clicks': 1}
{'banner_id': 2, 'shows': 15, 'clicks': 2}


ValueError: invalid literal for int() with base 10: ''

Какие средства для обработки ошибок существуют?

- Специальные возвращаемые значения (Golang)
```go
i, err := strconv.Atoi("42")
if err != nil {
    fmt.Printf("couldn't convert number: %v\n", err)
    return
}
fmt.Println("Converted integer:", i)
```

- Исключения (Python)

In [ ]:
int('abc')

- Исключения — специальный механизм языка для работы с ошибками.
- Прерывают нормальный ход исполнения программы.
- Сообщают о возникшей исключительной ситуации.
- Дают возможность обработать ошибку и восстановить работу программы.

Примеры исключений

In [ ]:
[0] * int(1e16)

Примеры исключений

In [ ]:
open('nonexistent.file')

Примеры исключений

In [ ]:
[1, 2, 3] + 4


Примеры исключений

In [ ]:
compile('a = 2 * 5 + 3)', '', 'exec')

Иерархия встроенных исключений: https://docs.python.org/3/library/exceptions.html#exception-hierarchy

```
BaseException
 ├── BaseExceptionGroup
 ├── GeneratorExit
 ├── KeyboardInterrupt
 ├── SystemExit
 └── Exception
      ├── ArithmeticError
      │    ├── FloatingPointError
      │    ├── OverflowError
      │    └── ZeroDivisionError
      ├── AssertionError
      ├── AttributeError
      ├── BufferError
      ├── EOFError
      ├── ExceptionGroup [BaseExceptionGroup]
      ├── ImportError
      │    └── ModuleNotFoundError
      ├── LookupError
      │    ├── IndexError
      │    └── KeyError
      ├── MemoryError
      ├── NameError
      │    └── UnboundLocalError
      ├── OSError
      │    ├── BlockingIOError
      │    ├── ChildProcessError
      │    ├── ConnectionError
      │    │    ├── BrokenPipeError
      │    │    ├── ConnectionAbortedError
      │    │    ├── ConnectionRefusedError
      │    │    └── ConnectionResetError
      │    ├── FileExistsError
      │    ├── FileNotFoundError
      │    ├── InterruptedError
      │    ├── IsADirectoryError
      │    ├── NotADirectoryError
      │    ├── PermissionError
      │    ├── ProcessLookupError
      │    └── TimeoutError
      ├── ReferenceError
      ├── RuntimeError
      │    ├── NotImplementedError
      │    └── RecursionError
      ├── StopAsyncIteration
      ├── StopIteration
      ├── SyntaxError
      │    └── IndentationError
      │         └── TabError
      ├── SystemError
      ├── TypeError
      ├── ValueError
      │    └── UnicodeError
      │         ├── UnicodeDecodeError
      │         ├── UnicodeEncodeError
      │         └── UnicodeTranslateError
      └── Warning
           ├── BytesWarning
           ├── DeprecationWarning
           ├── EncodingWarning
           ├── FutureWarning
           ├── ImportWarning
           ├── PendingDeprecationWarning
           ├── ResourceWarning
           ├── RuntimeWarning
           ├── SyntaxWarning
           ├── UnicodeWarning
           └── UserWarning
```

Обработка исключений: `try...except`

In [ ]:
filename = 'nonexistent.file'

try:
    fd = open(filename, 'r')
except FileNotFoundError:  # catch exceptions which satisfy isinstance(exc, FileNotFoundError)
    print(f'File {filename!r} does not exist')

Обработка исключений: `try...except...except`

In [ ]:
filename = 'nonexistent.file'

try:
    fd = open(filename, 'r')
except FileNotFoundError:
    print(f'File {filename!r} does not exist')
except (TypeError, ValueError, MemoryError) as e:
    print('Just to demonstrate a tuple of exceptions')
except Exception as e:  # the first matching except clause is triggered

    print(f'Exception occured while reading file {filename!r}: {e!r}')

Обработка исключений: `try...except...else...finally`

In [ ]:
f = None
try:
    f = open("filename.txt", 'r') # something dangerous
    1 / 0
except ValueError as e:  # scope failure
    print(f'Something bad happened: {e!r}')
else:  # scope success
    print('Nothing bad happened')
finally:  # scope exit
    if f is not None:
        f.close()
    print('Print this no matter what')

Стратегии обработки ошибок: **Look Before You Leap**

In [ ]:
def ctr(shows, clicks):
    """Returns banner click-through rate"""
    if shows == 0:
        return 0
    return clicks / shows

Стратегии обработки ошибок: **It's easier to ask for forgiveness than permission**

In [ ]:
def ctr(shows, clicks):
    """Returns banner click-through rate"""
    try:
        return clicks / shows
    except ZeroDivisionError:
        return 0

<div class="alert alert-danger">
<b>Антипаттерн: </b> Ловить BaseException
</div>

In [ ]:
try:
    do_dangerous()
except:  # catch everything, even KeyboardInterrupt
    pass

In [ ]:
try:
    do_dangerous()
except BaseException:  # same as above
    pass

Старайтесь максимально конкретизировать исключения в except

Бросить исключение можно с помощью ключевого слова `raise`

In [ ]:
raise ValueError('Positive integer expected')

Исключение должно быть объектом типа BaseException или его наследника

In [ ]:
raise 42

`raise` без аргумента перебрасывает последнее пойманное исключение.

In [ ]:
try:
    raise RuntimeError('Crash hard')
except Exception as e:
    print('Unknown error occured, no chance to recover, run!')
    raise

In [ ]:
raise

### Цепочки исключений

In [ ]:
def ctr(shows, clicks):
    """Returns banner click-through rate"""
    try:
        try:
            return clicks / shows
        except ZeroDivisionError as e:
            raise ValueError('Bad banner') from e
    except ValueError as e:
        raise OSError from e
        
ctr(0, 1)

### Причина исключения

`raise ... from ...`

In [ ]:
def ctr(shows, clicks):
    """Returns banner click-through rate"""
    try:
        return clicks / shows
    except ZeroDivisionError as e:
        raise ValueError('Bad banner') from e
try:
    ctr(0, 1)
except ValueError:
    print("catch value error")
except ZeroDivisionError:
    print("catch zero div")


### Сброс контекста

In [ ]:
def ctr(shows, clicks):
    """Returns banner click-through rate"""
    try:
        return clicks / shows
    except ZeroDivisionError as e:
        raise ValueError('Bad banner') from None
ctr(0, 1)

Можно создавать свои классы исключений, достаточно отнаследоваться от `Exception`. Хорошая практика — наследовать свои исключения от общего предка, чтобы их было удобнее ловить. Пример кастомных исключений: https://github.com/psf/requests/blob/master/requests/exceptions.py

In [ ]:
class ShoeError(Exception):
    pass

class WrongFootError(ShoeError):
    def __str__(self):
        return f'Try another one!'
        
raise WrongFootError([1, 2, 3])

Как устроены объекты-исключения

In [ ]:
try:
    raise ValueError(1, 2, 3)
except Exception as e:
    exc = e

In [ ]:
exc.args  # аргументы конструктора

In [ ]:
exc.__cause__  # причина исключения, устанавливается при raise EXC from CAUSE
exc.__context__  # последнее пойманное исключение, для цепочек исключений
exc.__traceback__

In [ ]:
#exc.with_traceback(tb)  # устанавливает __traceback__ в новое значение tb
exc.add_note("some text")
raise exc

### Warnings

In [ ]:
import numpy as np

np.int32(1) / np.int32(0)

In [ ]:
import numpy as np

try:
    np.int32(1) / np.int32(0)
except Exeption:
    print("Exception")

То есть не смотря на то, что Warnings - наследник Exeption, всё равно не удается поймать и обработать warning.

### Warnings

In [ ]:
import numpy as np
import warnings

warnings.filterwarnings("error")
try:
    np.int32(1) / np.int32(0)
except Exception as e:
    print(f"Exception {e!r}")

warnings.resetwarnings()

In [ ]:
import numpy as np
import warnings

warnings.filterwarnings("ignore")
try:
    np.int32(1) / np.int32(0)
except Exception as e:
    print(f"Exception {e!r}")

warnings.resetwarnings()

Полезные штуки

- `sys.exc_info()` — возвращает информацию о текущем обрабатываемом исключении
- Модуль `traceback`
- Модуль `warnings`

## Группы исключений

Проблема: предположим, что у нас есть следующая функция:

`run_parralel(list_of_functions, list_of_args, n_jobs=-1)`

Каждая вызванная внутри функция может выбросить ошибку. Какую ошибку тогда нужно ловить? Как понять, какие ошибки произошли в каждой из функций?

В чем проблемы Exception? Можно ли просто сложить exceptions в список? Почему Как должен быть устроен "полноценный" Exception, который способен поддерживать сложные исключения?

### ExceptionGroup

Exception не может в полной мере отразить независимый набор ошибок. Поэтому у нас есть ExceptionGroup, представляющий из себя дерево.

In [ ]:
eg = ExceptionGroup(
     "one",
     [
         TypeError(1),
         ExceptionGroup(
             "two",
              [TypeError(2), ValueError(3)]
         ),
         ExceptionGroup(
              "three",
               [OSError(4)]
         )
    ]
)
raise eg

### Методы обработки ExceptionGroup: subgroup

In [ ]:
type_errors = eg.subgroup(lambda e: isinstance(e, TypeError))
raise type_errors

### Методы обработки ExceptionGroup: split

In [ ]:
type_errors, other_errors = eg.split(lambda e: isinstance(e, TypeError))
raise other_errors

### Методы обработки ExceptionGroup: except*

In [ ]:
import errno

def low_level_os_operation() -> None:
    raise ExceptionGroup(
        "subtasks",
        [
            OSError(errno.EPIPE, "Broken pipe"),
            OSError(errno.ENOENT, "No such file or directory"),
            OSError(errno.EACCES, "Permission denied"),
            ValueError("bad value")
        ]
    )
    
try:
    low_level_os_operation()
except* OSError as errors:
    exc = errors.subgroup(lambda e: isinstance(e, OSError) and e.errno != errno.EPIPE)
    if exc is not None:
        raise exc from None
except* ValueError as errors:
    raise errors from None

# Менеджеры контекста

Начнём издалека.

Доклад Скотта Майерса "Why C++ Sails When the Vasa Sank" в Яндексе, 2014.

__"What you would consider the single most important feature in C++?"__

https://youtu.be/ltCgzYcpFUI?t=952

Как гарантировать, что некоторое действие будет выполнено вне зависимости от того, произошло исключение или нет?

In [ ]:
def do_something_dangerous(fd):
    raise RuntimeError('Not today!')

fd = open('myfile.txt', 'w')
try:
    do_something_dangerous(fd)
finally:
    print('Closing file')
    fd.close()
    print('File closed')

Менеджеры контекста предоставляют удобный способ провести инициализацию и гарантированную финализацию "контекста".

In [ ]:
r = aquire_resource()
try:
    use_resource(r)
finally:
    release_resource(r)

In [ ]:
with aquire_resource() as r:
    use_resource(r)

Примеры менеджеров контекста: `open`

In [ ]:
with open('filename.txt', 'w') as fd:
    fd.write("Hello")
# file is closed
fd.write("world")

Примеры менеджеров контекста: `tempfile`

In [ ]:
import tempfile

with tempfile.TemporaryFile() as tmp:
    do_something(tmp)
# tmp file is removed

Примеры менеджеров контекста: Python Database API

In [ ]:
import psycopg2
with psycopg2.connect(...) as conn:
    with conn.cursor() as cursor:
        cursor.execute('SELECT * FROM MyTable', params)
        result = cursor.fetchall()
# cursor.close() is called
# conn.commit() or conn.rollback() is called

Примеры менеджеров контекста: `pytest`

In [ ]:
import pytest
with pytest.raises(ZeroDivisionError):
    a = 1 / 0
# ZeroDivisionError is not expected to occur anymore and will cause test to fail
with pytest.raises(ZeroDivisionError):
    a = 0 / 1

Примеры менеджеров контекста: `warnings`

In [ ]:
import numpy as np
import warnings

with warnings.catch_warnings(record=True) as w:
    # Cause all warnings to always be triggered.
    warnings.simplefilter("always")
    np.int32(1) / np.int32(0)
    np.log(0)
    
    for warn in w:
        print(warn)

Синтаксис выражения `with`

In [ ]:
# nested contexts
with open('file1.txt') as file1, open('file2.txt') as file2:
    do_something(f, s)
# since python 3.11
with (open('file1.txt'), open('file2.txt')) as (file1, file2):
    do_something(f, s)

In [ ]:
# same as above
with first() as f:
    with second as s():
        do_something(f, s)

In [ ]:
with third():  # <as NAME> part as optional
    do_something()

Менеджеры контекста — объекты, реализующие специальный протокол

In [ ]:
import typing as tp
from types import TracebackType

class MyContextManager:
    def __enter__(self) -> tp.Self: # with () as X
        # initialize context
        return self
    
    def __exit__(self,
                 exc_type: type[BaseException] | None,
                 exc_value: BaseException | None,
                 traceback: TracebackType | None) -> bool | None:
        # finalize context
        if exc_value is not None:
            return True  # return True from __exit__ to suppress the exception

Семантика

In [ ]:
with acquire_resource() as resource:
    use_resource(resource)

In [ ]:
manager = acquire_resource()
resource = manager.__enter__()
try:
    use_resource(resource)
finally:
    exc_type, exc_value, traceback = sys.exc_info()
    suppress = manager.__exit__(exc_type, exc_value, traceback)
    if exc_value is not None and not suppress:
        raise exc_value

Полушуточный пример

In [ ]:
class Tag:
    def __init__(self, name):
        self.name = name
    def __enter__(self):
        print('<{}>'.format(self.name))
        return self
    
    def __exit__(self, *args):
        print('</{}>'.format(self.name))
        
    def update(self, new_name):
        self.name = new_name

        
with Tag('table') as table:
    table.update("broken_table")
    with Tag('tr'):
        with Tag('td'):
            print('cell 1')
        with Tag('td'):
            print('cell 2')

`contextlib.contextmanager` — удобный способ создавать менеджеры контекста

In [ ]:
from contextlib import contextmanager

@contextmanager
def mycm():
    print('before')
    yield 42  # yep, it is a generator
    print('after')
    
with mycm() as r:
    print(f'got {r}')
    
with mycm() as r:
    raise RuntimeError('Oops')
# 'after' is not printed!

Но работать с `contextlib.contextmanager` надо аккуратно

In [ ]:
from contextlib import contextmanager

@contextmanager
def mycm():
    print('before')
    try:
        yield ValueError("kek")
    finally:
        print('after')

with mycm() as r:
    print(type(r), r)
    raise RuntimeError('Oops')

В модуле `contextlib` есть и другие полезные штуки:
- `contextlib.ContextDecorator` — базовый класс для менеджеров контекста, их потом можно будет использовать как декораторы для функций
- `contextlib.ExitStack` — позволяет использовать неизвестное заранее количество "ресурсов", динамически управлять менеджерами контекста
- См. документацию